# Seminario de Invierno. CAPAP-H
# Cáceres. 29-31 Enero 2025

#### Juan Mario Haut Hurtado. Correo: juanmariohaut@unex.es

In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

### Definición del modelo

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



training_data = datasets.MNIST(
    root="/tmp",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="/tmp",
    train=False,
    download=True,
    transform=ToTensor()
)

dataloader_tr = DataLoader(training_data, batch_size=128, shuffle=True)
dataloader_te = DataLoader(test_data, batch_size=1280, shuffle=False)

In [6]:
modelo = Net()

In [13]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    log_interval = 10
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    log_interval = 10
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
optimizer = optim.Adadelta(modelo.parameters(), lr=1.0)
n_epoch = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

modelo.to(device)

for i in range(n_epoch):
    train(modelo, device, dataloader_tr, optimizer, i)
    test(modelo, device, dataloader_te)

In [2]:
# Terminar de entrenar el modelo